<center><img src="http://alacip.org/wp-content/uploads/2014/03/logoEscalacip1.png" width="500"></center>


<center> <h1>Curso: Introducción al Python</h1> </center>

<br></br>

* Profesor:  <a href="http://www.pucp.edu.pe/profesor/jose-manuel-magallanes/" target="_blank">Dr. José Manuel Magallanes, PhD</a> ([jmagallanes@pucp.edu.pe](mailto:jmagallanes@pucp.edu.pe))<br>
    - Profesor del **Departamento de Ciencias Sociales, Pontificia Universidad Católica del Peru**.<br>
    - Senior Data Scientist del **eScience Institute** and Visiting Professor at **Evans School of Public Policy and Governance, University of Washington**.<br>
    - Fellow Catalyst, **Berkeley Initiative for Transparency in Social Sciences, UC Berkeley**.


## Parte 5:  Formateando Valores en Python

<a id='beginning'></a>

Una vez que los datos han sido limpiados, es momento de ver si tienen el formato para ser tratados estadísticamente, es decir, que representan el correcto tipo de [escala](#scale) y que representen una adecuada [monotonía](#monotony).

Traigamos unos datos conocidos, pero acelerando la limpieza:

In [1]:
wikiLink="https://en.wikipedia.org/wiki/List_of_freedom_indices" 

#traer tabla
import pandas as pd
DF=pd.read_html(wikiLink,header=0,flavor='bs4',attrs={'class': 'wikitable sortable',})[0]

#limpiando celdas
import re  
pattern='\\s+|\\d+|\\[|\\]'
nothing=''
DF.columns=[re.sub(pattern,nothing,element) for element in DF.columns]
DF.head()

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Afghanistan,not free,mostly unfree,difficult situation,authoritarian regime
1,Albania,partly free,moderately free,noticeable problems,hybrid regime
2,Algeria,not free,repressed,difficult situation,authoritarian regime
3,Andorra,free,NaN,satisfactory situation,NaN
4,Angola,not free,mostly unfree,noticeable problems,authoritarian regime


____
<a id='scale'></a>

### Las escalas de medición

Para saber si están en la escala correcta, debemos usar _dtypes_:

In [2]:
DF.dtypes

Country                   object
FreedomintheWorld         object
IndexofEconomicFreedom    object
PressFreedomIndex         object
DemocracyIndex            object
dtype: object

Los cuatro indices son categorías, no texto (_object_). Hagamos la conversión:

In [3]:
headers=DF.columns
DF[headers[1:]]=DF[headers[1:]].astype('category')

In [4]:
# sale:
DF.dtypes

Country                     object
FreedomintheWorld         category
IndexofEconomicFreedom    category
PressFreedomIndex         category
DemocracyIndex            category
dtype: object

Este cambio es imperceptible a la vista:

In [5]:
DF.head()

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Afghanistan,not free,mostly unfree,difficult situation,authoritarian regime
1,Albania,partly free,moderately free,noticeable problems,hybrid regime
2,Algeria,not free,repressed,difficult situation,authoritarian regime
3,Andorra,free,NaN,satisfactory situation,NaN
4,Angola,not free,mostly unfree,noticeable problems,authoritarian regime


Ahora si podemos saber sus valores:

In [6]:
DF.FreedomintheWorld.cat.categories

Index(['free', 'not free', 'partly free'], dtype='object')

In [7]:
DF.IndexofEconomicFreedom.cat.categories

Index(['free', 'moderately free', 'mostly free', 'mostly unfree', 'repressed'], dtype='object')

In [8]:
DF.PressFreedomIndex.cat.categories

Index(['difficult situation', 'good situation', 'noticeable problems',
       'satisfactory situation', 'very serious situation'],
      dtype='object')

In [9]:
DF.DemocracyIndex.cat.categories

Index(['authoritarian regime', 'flawed democracy', 'full democracy',
       'hybrid regime'],
      dtype='object')

Vemos que tenemos hasta 5 niveles en 2 variables, y 3 y 4 niveles en otras. De ahi que lo prudente es encontrar la distribución común de valores que refleja la ordinalidad, y los máximos y mínimos. 

Veamos como volverla ordinal. Primero los valores:

In [10]:
oldFree=list(DF.FreedomintheWorld.cat.categories)
newFree=['very good','very bad','middle']
recodeFree={old:new for old,new in zip (oldFree,newFree)}

oldEco=list(DF.IndexofEconomicFreedom.cat.categories)
newEco=['very good','middle','good','bad','very bad']
recodeEco={old:new for old,new in zip (oldEco,newEco)}


oldPress=list(DF.PressFreedomIndex.cat.categories)
newPress=['bad','very good','middle','good','very bad']
recodePress={old:new for old,new in zip (oldPress,newPress)}

oldDemo=list(DF.DemocracyIndex.cat.categories)
newDemo=['very bad','good','very good','bad']
recodeDemo={old:new for old,new in zip (oldDemo,newDemo)}

Ahora usamos los dicts creado para recodificar:

In [11]:
DF.FreedomintheWorld.cat.rename_categories(recodeFree,inplace=True)

DF.IndexofEconomicFreedom.cat.rename_categories(recodeEco,inplace=True)

DF.PressFreedomIndex.cat.rename_categories(recodePress,inplace=True)

DF.DemocracyIndex.cat.rename_categories(recodeDemo,inplace=True)

# veamos:
DF

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Afghanistan,very bad,bad,bad,very bad
1,Albania,middle,middle,middle,bad
2,Algeria,very bad,very bad,bad,very bad
3,Andorra,very good,NaN,good,NaN
4,Angola,very bad,bad,middle,very bad
5,Antigua and Barbuda,very good,NaN,middle,NaN
6,Argentina,very good,bad,middle,good
7,Armenia,middle,middle,middle,bad
8,Australia,very good,very good,good,very good
9,Austria,very good,good,good,very good


Los datos aun no son ordinales, pero aqui serán:

In [12]:
from pandas.api.types import CategoricalDtype


ordinal = CategoricalDtype(categories=['very good','good','middle','bad','very bad'],ordered=True)

to_Order=lambda x: x.astype(ordinal)
DF[headers[1:]]=DF[headers[1:]].apply(to_Order)

In [13]:
# asi va:
DF.head()

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Afghanistan,very bad,bad,bad,very bad
1,Albania,middle,middle,middle,bad
2,Algeria,very bad,very bad,bad,very bad
3,Andorra,very good,NaN,good,NaN
4,Angola,very bad,bad,middle,very bad


Notemos que las modalidades no usadas están presentes:

In [14]:
DF.FreedomintheWorld.value_counts(sort=False,dropna=False)

very good    87
good          0
middle       62
bad           0
very bad     55
NaN           1
Name: FreedomintheWorld, dtype: int64

Verificaciones adicionales:

In [15]:
#las categorias:
DF.PressFreedomIndex.cat.categories

Index(['very good', 'good', 'middle', 'bad', 'very bad'], dtype='object')

In [16]:
#tipo de escala?
DF.PressFreedomIndex.cat.ordered

True

[Go to page beginning](#beginning)
____
<a id='monotony'></a>

### Cambio de Monotonía:

Verifiquemos si está bien la asignación que hemos hecho:

In [17]:
DF.PressFreedomIndex.head()

0       bad
1    middle
2       bad
3      good
4    middle
Name: PressFreedomIndex, dtype: category
Categories (5, object): [very good < good < middle < bad < very bad]

In [18]:
DF.PressFreedomIndex.max()

'very bad'

Este es un caso donde quiza la intensidad creciente debe ser hacia el sentido positivo del concepto. Claro que pudimos hacerlo al inicio, pero aprovechemos para saber cómo se hace.

Para ello crearé una función:

In [19]:
def changeMonotony(aColumn):
    # Invierto las categorias:
    newOrder= aColumn.cat.categories[::-1]  # [::-1]  reverses the list.
    # aplico función
    return aColumn.cat.reorder_categories(newOrder,ordered=True)

Esta función la aplica de nuevo, columna por columna:

In [20]:
# SOLO UNA VEZ!! (sino reintenta)
DF[headers[1:]]=DF[headers[1:]].apply(changeMonotony)

¿Funcionó?

In [21]:
DF.PressFreedomIndex.head(20)

0           bad
1        middle
2           bad
3          good
4        middle
5        middle
6        middle
7        middle
8          good
9          good
10     very bad
11       middle
12     very bad
13          bad
14       middle
15          bad
16    very good
17       middle
18       middle
19       middle
Name: PressFreedomIndex, dtype: category
Categories (5, object): [very bad < bad < middle < good < very good]

In [22]:
DF.PressFreedomIndex.max()

'very good'

Todo lo que hemos trabajado podríamos entregarselo a R para que haga su trabajo estadístico, pero como no tiene metadata, es mejor guardar los indices ordinales como número:

In [23]:
oldlevels=['very bad','bad','middle','good','very good']
newlevels=[1,2,3,4,5]
recodeLevels={old:new for old,new in zip (oldlevels,newlevels)}

In [24]:
renamer=lambda column: column.cat.rename_categories(recodeLevels)
DF[headers[1:]]=DF[headers[1:]].apply(renamer)
DF.head(10)

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Afghanistan,1,2.0,2,1.0
1,Albania,3,3.0,3,2.0
2,Algeria,1,1.0,2,1.0
3,Andorra,5,NaN,4,NaN
4,Angola,1,2.0,3,1.0
5,Antigua and Barbuda,5,NaN,3,NaN
6,Argentina,5,2.0,3,4.0
7,Armenia,3,3.0,3,2.0
8,Australia,5,5.0,4,5.0
9,Austria,5,4.0,4,5.0


Un tema adicional son los valores perdidos. Hay varios NaN.

La función para reemplazarlos es sencilla, pero hay que evitar facilismos. Veamos:

In [25]:
#recordar:
DF.dtypes

Country                     object
FreedomintheWorld         category
IndexofEconomicFreedom    category
PressFreedomIndex         category
DemocracyIndex            category
dtype: object

In [26]:
#tienen que ser numericos:
DF[headers[1:]]=DF[headers[1:]].apply(pd.to_numeric)

In [27]:
#mediana por grupos: 
DF.groupby(headers[1])[headers[2:]].median()

,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
FreedomintheWorld,,,
1.0,2.0,2.0,1.0
3.0,2.0,3.0,2.0
5.0,3.0,4.0,4.0


Hemos calculado la mediana de cada indice que no sea Freedom in the world, pues ésta sólo tiene 1 valor perdido:


In [28]:
DF.info() #206 buenos de 207

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
Country                   205 non-null object
FreedomintheWorld         204 non-null float64
IndexofEconomicFreedom    180 non-null float64
PressFreedomIndex         189 non-null float64
DemocracyIndex            167 non-null float64
dtypes: float64(4), object(1)
memory usage: 8.1+ KB


In [29]:
# o
DF.isnull().sum() 

Country                    0
FreedomintheWorld          1
IndexofEconomicFreedom    25
PressFreedomIndex         16
DemocracyIndex            38
dtype: int64

In [30]:
import numpy as np

for h in headers[2:]:
    DF[h].fillna(DF.groupby(["FreedomintheWorld"])[h].transform(np.median), inplace=True)

Obteniendo:

In [31]:
DF.head(20)

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Afghanistan,1.0,2.0,2.0,1.0
1,Albania,3.0,3.0,3.0,2.0
2,Algeria,1.0,1.0,2.0,1.0
3,Andorra,5.0,3.0,4.0,4.0
4,Angola,1.0,2.0,3.0,1.0
5,Antigua and Barbuda,5.0,3.0,3.0,4.0
6,Argentina,5.0,2.0,3.0,4.0
7,Armenia,3.0,3.0,3.0,2.0
8,Australia,5.0,5.0,4.0,5.0
9,Austria,5.0,4.0,4.0,5.0


Un detalle pequeño es enviar esta data con buenos nombres de columnas:

In [32]:
DF.columns=["Country","WorldFreedom","EconomicFreedom","PressFreedom","Democracy"]

**Guardando archivo**

A esta altura es bueno guardar el archivo, pues ya está listo:

In [33]:
DF.to_csv("indexes.csv",index=None)

Traigamos la data de los departamentos de Colombia que vimos al final de la unidad anterior:

In [34]:
idhCol2='https://es.wikipedia.org/wiki/Anexo:Departamentos_de_Colombia_por_IDH'
idhColT2=pd.read_html(idhCol2,header=0,flavor='bs4',attrs={'class': 'sortable',},
                       thousands='\xa0', decimal=',')[0]
idhColT2.iloc[:,2]=idhColT2.iloc[:,2].str.replace("\s","")
idhColT2.columns=[element.split('[')[0].replace(" ","") for element in idhColT2.columns]
idhColT2.Entidad=[element.split('[')[0] for element in idhColT2.Entidad]
newRows=['Amazonas', 'Guainia', 'Guaviare', 'Vaupés', 'Vichada']
info=idhColT2[idhColT2.Entidad=='Región Amazónica'].values.tolist()[0][1:]
newData = pd.DataFrame([[row] + info for row in newRows], columns=idhColT2.columns)
idhColT2=idhColT2.append(newData,ignore_index=True)
idhColT2.drop([0,24,29],inplace=True)
numericos=list(idhColT2.IDH)
numericos.extend(list(idhColT2.Población))

inapropiados=[]
for n in numericos:
    try:
        float(n)
    except ValueError:
        if not n in inapropiados: # evitar duplicados
            inapropiados.append(n)
idhColT2.replace(inapropiados,value=[None]*len(inapropiados),inplace=True)
idhColT2.reset_index(drop=True,inplace=True)

IndexError: list index out of range

Así que actualmente, tenemos:

In [ ]:
idhColT2

Aquí el problema es distinto. Los datos faltantes necesitamos reemplazarlos, no estimarlos. Es decir, hay que traer la data de otro sitio.

In [35]:
import IPython

pobCol='https://es.wikipedia.org/wiki/Anexo:Departamentos_de_Colombia_por_poblaci%C3%B3n'
iframe = '<iframe src=' + pobCol + ' width=700 height=350></iframe>'
IPython.display.HTML(iframe)

In [36]:

colPOB=pd.read_html(pobCol,header=0,flavor='bs4',attrs={'class': 'sortable',},
                       thousands='\xa0', decimal=',')[0]

# q ha venido?
colPOB.dtypes

N.º                   float64
Departamento           object
Población Cabecera      int64
Población Resto         int64
Población Total         int64
dtype: object

In [37]:
colPOB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 5 columns):
N.º                   33 non-null float64
Departamento          34 non-null object
Población Cabecera    34 non-null int64
Población Resto       34 non-null int64
Población Total       34 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.4+ KB


Recuerda que tenemos 32 regiones, es decir aquí ha venido algo extra: Bogotá y Colombia:

In [38]:
colPOB[colPOB.Departamento.isin (['Colombia','Bogotá','Cundinamarca'])]

,N.º,Departamento,Población Cabecera,Población Resto,Población Total
0,1.0,Bogotá,8164178,16869,8181047
3,4.0,Cundinamarca,1906623,897615,2804238
33,NaN,Colombia,38295351,11538889,49834727


De aquí, vemos que a la info de Cundinamarca debe sumarsele la de Bogotá, y eliminar luego Bogotá y Colombia.

In [39]:
colPOB.iloc[3,2:]=colPOB.iloc[3,2:]+colPOB.iloc[0,2:]

In [40]:
colPOB.drop([0,33],inplace=True)
colPOB.reset_index(drop=True,inplace=True)

In [41]:
# asi queda:
colPOB

,N.º,Departamento,Población Cabecera,Población Resto,Población Total
0,2.0,Antioquia,5262135,1428842,6690977
1,3.0,Valle del Cauca,4169228,586472,4755760
2,4.0,Cundinamarca,10070801,914484,10985285
3,5.0,Atlántico,2438772,107366,2546138
4,6.0,Bolívar,1693970,477588,2171558
5,7.0,Santander,1588034,502820,2090854
6,8.0,Nariño,907725,901576,1809301
7,9.0,Córdoba,950230,838418,1788648
8,10.0,Tolima,980713,439244,1419957
9,11.0,Cauca,567610,848535,1416145


Preparemonos para llevar esta info a la data anterior.

Eliminemos cosas innecesarias. La primera columna (No) no es necesaria:

In [42]:
colPOB.drop(['N.º'], axis=1,inplace=True)

En la data con el IDH por provincia, Población y PaísComparable tampoco son necesarias:

In [43]:
idhColT2.drop(['Población','PaísComparable'], axis=1,inplace=True)

Ambas datas tienen el mismo tamaño?

In [44]:
len(colPOB)==len(idhColT2)

False

Si es así, la unión de ambas debería ser igual, considerando lo que hemos hecho, pero siempre hay detalles que faltan:

In [45]:
test=idhColT2.merge(colPOB,left_on='Entidad',right_on='Departamento',how='outer')
test

,Entidad,IDH,Departamento,Población Cabecera,Población Resto,Población Total
0,Bogotá,0.792,NaN,NaN,NaN,NaN
1,Valle del Cauca,0.771,Valle del Cauca,4169228.0,586472.0,4755760.0
2,San Andrés y Providencia,0.770,San Andrés y Providencia,56487.0,21926.0,78413.0
3,Atlántico,0.766,Atlántico,2438772.0,107366.0,2546138.0
4,Quindío,0.765,Quindío,506238.0,68722.0,574960.0
5,Meta,0.758,Meta,775621.0,241051.0,1016672.0
6,Santander,0.758,Santander,1588034.0,502820.0,2090854.0
7,Caldas,0.757,Caldas,719730.0,274140.0,993870.0
8,Cundinamarca,0.754,Cundinamarca,10070801.0,914484.0,10985285.0
9,Antioquia,0.752,Antioquia,5262135.0,1428842.0,6690977.0


Nuestro test nos muestra quienes no concuerdan para el __merge__:

In [46]:
test[pd.isnull(test.IDH) | pd.isnull(test.Departamento)]

,Entidad,IDH,Departamento,Población Cabecera,Población Resto,Población Total
0,Bogotá,0.792,NaN,NaN,NaN,NaN
33,Colombia,0.748,NaN,NaN,NaN,NaN


Arriba se nota por que no hubo una combinación perfecta. Resolvamos y re hagamos:

In [47]:
idhColT2[idhColT2.Entidad.isin(['San Andrés', 'Guainia'])]


,Entidad,IDH


In [48]:
colPOB[colPOB.Departamento.isin(['San Andrés y Providencia', 'Guainía'])]

,Departamento,Población Cabecera,Población Resto,Población Total
28,San Andrés y Providencia,56487,21926,78413
31,Guainía,13090,30356,43446


In [49]:
idhColT2.loc[8, 'Entidad']=colPOB.loc[28, 'Departamento']
idhColT2.loc[28, 'Entidad']=colPOB.loc[31, 'Departamento']

Ahora debe estar bien:

In [50]:
idhColT2.merge(colPOB,left_on='Entidad',right_on='Departamento',how='outer')

,Entidad,IDH,Departamento,Población Cabecera,Población Resto,Población Total
0,Bogotá,0.792,NaN,NaN,NaN,NaN
1,Valle del Cauca,0.771,Valle del Cauca,4169228.0,586472.0,4755760.0
2,San Andrés y Providencia,0.770,San Andrés y Providencia,56487.0,21926.0,78413.0
3,San Andrés y Providencia,0.754,San Andrés y Providencia,56487.0,21926.0,78413.0
4,Atlántico,0.766,Atlántico,2438772.0,107366.0,2546138.0
5,Quindío,0.765,Quindío,506238.0,68722.0,574960.0
6,Meta,0.758,Meta,775621.0,241051.0,1016672.0
7,Santander,0.758,Santander,1588034.0,502820.0,2090854.0
8,Caldas,0.757,Caldas,719730.0,274140.0,993870.0
9,Antioquia,0.752,Antioquia,5262135.0,1428842.0,6690977.0


Hagamos el merge y eliminemos Entidad:

In [51]:
idhColFinal=idhColT2.merge(colPOB,left_on='Entidad',right_on='Departamento',how='outer')
idhColFinal.drop(['Entidad'],axis=1,inplace=True)
idhColFinal

,IDH,Departamento,Población Cabecera,Población Resto,Población Total
0,0.792,NaN,NaN,NaN,NaN
1,0.771,Valle del Cauca,4169228.0,586472.0,4755760.0
2,0.770,San Andrés y Providencia,56487.0,21926.0,78413.0
3,0.754,San Andrés y Providencia,56487.0,21926.0,78413.0
4,0.766,Atlántico,2438772.0,107366.0,2546138.0
5,0.765,Quindío,506238.0,68722.0,574960.0
6,0.758,Meta,775621.0,241051.0,1016672.0
7,0.758,Santander,1588034.0,502820.0,2090854.0
8,0.757,Caldas,719730.0,274140.0,993870.0
9,0.752,Antioquia,5262135.0,1428842.0,6690977.0


Hemos visto que el lenguaje español añade pequeñas complejidades, pues usa tildes. Creemos una columna _normalizada_ extra con el nombre del departamento:

In [52]:
#instalar unidecode
from unidecode import unidecode as notilde

ModuleNotFoundError: No module named 'unidecode'

In [53]:
byeTilde=lambda x: x if x is None else notilde(x)
idhColFinal[['DepartamentoNorm']]=idhColFinal[['Departamento']].applymap(byeTilde)
#
idhColFinal

NameError: ("name 'notilde' is not defined", 'occurred at index Departamento')

Al igual que en el caso anterior, debemos a esta altura guardar nuestro archivo:

In [54]:
idhColFinal.to_csv("colombia.csv",index=None)

_____

**AUSPICIO**: 

* El desarrollo de estos contenidos ha sido posible gracias al grant del Berkeley Initiative for Transparency in the Social Sciences (BITSS) at the Center for Effective Global Action (CEGA) at the University of California, Berkeley


<center>
<img src="https://www.bitss.org/wp-content/uploads/2015/07/bitss-55a55026v1_site_icon.png" style="width: 200px;"/>
</center>

* Este curso cuenta con el auspicio de:


<center>
<img src="https://www.python.org/static/img/psf-logo@2x.png" style="width: 500px;"/>
</center>



**RECONOCIMIENTO**


EL Dr. Magallanes agradece a la Pontificia Universidad Católica del Perú, por su apoyo en la participación en la Escuela ALACIP.

<center>
<img src="https://dci.pucp.edu.pe/wp-content/uploads/2014/02/Logotipo_colores-290x145.jpg" style="width: 400px;"/>
</center>


El autor reconoce el apoyo que el eScience Institute de la Universidad de Washington le ha brindado desde el 2015 para desarrollar su investigación en Ciencia de Datos.

<center>
<img src="https://escience.washington.edu/wp-content/uploads/2015/10/eScience_Logo_HR.png" style="width: 500px;"/>
</center>

<br>
<br>